In [88]:
""" minimalistic adiabatic parcel model using Numpy, pint, SciPy/odeint and matplotlib """
import math, pint, scipy, chempy, ipywidgets, IPython
import numpy as np
from matplotlib import pyplot
si = pint.UnitRegistry()
si.setup_matplotlib()

In [89]:
IDX_S = -1
IDX_T = -2

In [90]:
def sample_spectrum(parameters):
    spectrum = scipy.stats.lognorm(*(
        math.log(parameters['lognormal_geometric_stdev']),
        0,
        parameters['lognormal_median_radius'].to(si.m).magnitude
    ))
    n_sd = parameters['n_sd']
    rd_magnitude = [spectrum.ppf((2 * i - 1) / (2 * n_sd)) for i in range(1, n_sd + 1)]
    rd_unit = si.m
    sampled_values = {
        'multiplicity': [parameters['total_aerosol_number'] / n_sd] * n_sd,
        'dry radius': [rd * rd_unit for rd in rd_magnitude]
    }
    pyplot.plot(rd_magnitude * si.m,spectrum.pdf(rd_magnitude) / rd_unit)
    pyplot.step(
        (rd_magnitude[:-1] + np.diff(rd_magnitude)/2) * rd_unit,
        1 / n_sd / np.diff(rd_magnitude) / rd_unit ,
        where='mid',
        label='sampled distribution'
    )
    pyplot.gca().xaxis.set_units(si.um)
    pyplot.gca().yaxis.set_units(si.um**-1)
    pyplot.legend()

    pyplot.grid(True)
    pyplot.show()
    return sampled_values

CONST = {
    'M_dry': (
        0.76 * chempy.Substance.from_formula("N2").mass * si.gram / si.mole +
        0.23 * chempy.Substance.from_formula("O2").mass * si.gram / si.mole +
        0.01 * chempy.Substance.from_formula("Ar").mass * si.gram / si.mole
    ),
    'M_vap': chempy.Substance.from_formula("H2O").mass * si.gram / si.mole,
    'g': scipy.constants.g * si.m / si.s**2,
    'water_density': 1000 * si.kg / si.m**3,
    "specific_heat_constant_pressure": 1005 * si.J / si.kg / si.K,
    "latent_heat": 2.27e6 * si.J / si.kg,
    "surface_tension_water": 7.5e-2 * si.N / si.m,
}
CONST['R_dry'] = scipy.constants.R * si.J / si.mole / si.K / CONST['M_dry']
CONST['eps'] = CONST['M_vap'] / CONST['M_dry']
CONST['R_vap']  =  CONST['R_dry'] / CONST['eps']

In [91]:
def saturated_vapor_pressure(temperature):
    """ eq. 2.17 in Rogers & Yau """
    return 6.112 * si.hPa * math.exp(
        17.67 * temperature / (temperature + 243.5 * si.K)
    )

def equil_saturation(radius, kappa, dry_radius, temperature):
    """ eq. 6.6 in R&Y """
    drop_surface_term = (2 * CONST['surface_tension_water']) / (
            CONST['R_dry'] * CONST['water_density'] * temperature)
    drop_solute_term = kappa * dry_radius**3
    return 1 + drop_surface_term/radius - drop_solute_term/radius**3

def funct(state, _, parameters):
    """ returns time derivative of the state vector """
    saturation = state[IDX_S]
    temperature = state[IDX_T] * parameters['units'][IDX_T]
    condensation_rate_constant = (CONST['water_density'] *
                                  4 * math.pi /
                                  parameters['total_aerosol_number'] /
                                  parameters['mass_of_dry_air'])

    water_vapor_diffusivity = 1e-5 * si.m**2 / si.s * (
            (0.015 / si.K * temperature) - 1.9)
    thermal_conductivity = (
            (1.5e-11 * si.W / si.m / si.K**4) * temperature**3
            + (-4.8e-8 * si.W / si.m / si.K**3) * temperature**2
            + (1e-4 * si.W / si.m / si.K**2) * temperature
            + -3.9e-4 * si.W / si.m / si.K
        )

    p_vs = saturated_vapor_pressure(temperature)
    vapour_pressure = saturation * p_vs
    
    vapour_density = vapour_pressure / (CONST['R_vap'] * temperature)
    a_coeff = (CONST['g'] / (CONST['R_dry'] * temperature) *
               ((CONST['eps'] * CONST["latent_heat"]) / (CONST["specific_heat_constant_pressure"] * temperature) - 1 )
    )  # equation 7.23

    a_value = ((p_vs * water_vapor_diffusivity)
               / (CONST['water_density'] * temperature * CONST['R_dry']))
    b_value = (((thermal_conductivity * temperature)/ (CONST['water_density'] * CONST["latent_heat"]))
               * ((CONST["latent_heat"] / (CONST['R_dry'] * temperature)) - 1))

    g_term = a_value + b_value

    deriv = [None] * len(state)

    condensation_rate = 0 / si.s
    drop_mass = 0 * si.kg
    for i in range(parameters['n_sd']):
        radius = state[i]* parameters['units'][i]
        multiplicity = parameters['sampled_spectrum']['multiplicity'][i]
        
        drop_mass += multiplicity * CONST['water_density'] * 4/3 * np.pi * radius**3

        saturation_eq = equil_saturation(
            radius = radius,
            kappa = parameters['kappa'],
            dry_radius = parameters['sampled_spectrum']['dry radius'][i],
            temperature = temperature
        )
        deriv[i] = g_term/radius * (saturation - saturation_eq) / parameters["updraft"]# equation 7.18

        condensation_rate += (condensation_rate_constant *
                              (multiplicity * radius**2 * deriv[i] * parameters["updraft"])
                             )
    mixing_ratio = ((parameters["mass of vapour at t=0"] -
                    drop_mass + parameters["mass of droplets at t=0"]) /
                    parameters['mass_of_dry_air'])

    dry_air_density = vapour_density / mixing_ratio
    total_density = vapour_density + dry_air_density

    pressure = vapour_pressure + (dry_air_density * temperature * CONST['R_dry'])
    
    b_coeff = total_density * (
        (CONST['R_dry'] * temperature / (CONST['eps'] * p_vs))
        +
        (CONST['eps'] * CONST["latent_heat"] ** 2 /
         (pressure * temperature * CONST["specific_heat_constant_pressure"])
         )
    )  # equation 7.24
    
    deriv[IDX_S] = ((a_coeff * parameters['updraft']  +
                    b_coeff * condensation_rate) /
                    parameters["updraft"]) # equation 7.22
    deriv[IDX_T] = (- ((CONST["latent_heat"] / CONST["specific_heat_constant_pressure"])
                    * condensation_rate) - ((CONST['g'] / CONST["specific_heat_constant_pressure"])
                    * parameters["updraft"])) / parameters["updraft"]

    return [
        deriv[i].to(parameters['units'][i] / si.m).magnitude
        for i in range(len(parameters['units']))
    ]

In [92]:
def compute_derived_parameters(parameters, initial_condition):
    p_vs = saturated_vapor_pressure(initial_condition[IDX_T])
    parameters["mass of vapour at t=0"] = ((
                                            (CONST['eps'] * initial_condition[IDX_S]) /
                                            ((parameters['initial total pressure'] /
                                              p_vs)- 1)) *
                                            parameters["mass_of_dry_air"])
    parameters["mass of droplets at t=0"] = 0 

    parameters['units'] = [None] * (parameters['n_sd'] + 2)
    parameters['units'][IDX_S] = si.dimensionless
    parameters['units'][IDX_T] = si.K
    parameters['units'][:parameters['n_sd']] = [si.m] * parameters['n_sd']   

def integrate(parameters):    
    initial_condition = [None] * (parameters['n_sd'] + 2)
    initial_condition[IDX_S] = parameters['initial relative humidity']
    initial_condition[:parameters['n_sd']] = [
        scipy.optimize.root_scalar(
            f=lambda r: equil_saturation(
                r * si.m,
                parameters['kappa'],
                parameters['sampled_spectrum']['dry radius'][i],
                parameters['initial temperature']
            ),
            x0=parameters['sampled_spectrum']['dry radius'][i]
        ).root * si.m
        for i in range(parameters['n_sd'])
    ]
    initial_condition[IDX_T] = parameters['initial temperature']
    
    compute_derived_parameters(parameters, initial_condition)
    
    nt = 40
    dt_out = 5 * si.s

    displacement = [(parameters["updraft"] * i * dt_out).to(si.m).magnitude for i in range(nt + 1)]
    
    integration_result = scipy.integrate.odeint(
        funct,
        [x.to(parameters['units'][i]).magnitude 
        for i, x in enumerate(initial_condition)],
        [z for z in displacement],
        args=(parameters,)
    )
    return displacement * si.m, [
        integration_result[:, i] * parameters['units'][i]
        for i in range(integration_result.shape[1])
    ]

In [93]:
def plot(displacement, integration_result_with_units):
    n_sd = len(integration_result_with_units) - 2
    PLOTS = (
        ((IDX_S,), "saturation", None),
        (range(n_sd), "drop radii", si.um),
        ((IDX_T,), "temperature", None),
        
    )
    
    _, axs = pyplot.subplots(1, len(PLOTS), sharey=True)
    
    for i, (vars, name, unit) in enumerate(PLOTS):
        for idx in vars:
            axs[i].plot(
                integration_result_with_units[idx],
                displacement,
            )
        axs[i].set_title(f"{name} vs. height")
        if unit: axs[i].xaxis.set_units(unit)
    
    for ax in axs:
        ax.grid()

In [94]:
output = ipywidgets.Output()

def parcel(kappa, updraft_m_s, n_sd):
    with output:
        IPython.display.clear_output(wait=True)
        
        parameters = {
            'updraft':  updraft_m_s * si.m / si.s,
            'kappa': kappa,
            'total_aerosol_number': 1e6,
            'lognormal_geometric_stdev': 1.2,
            'lognormal_median_radius': .05 * si.um,
            'mass_of_dry_air': 1 * si.kg,
            'initial total pressure': 900 *si.hPa,
            'initial relative humidity': .05 * si.dimensionless,
            'initial temperature': 233 * si.K,
            'n_sd': n_sd
        }
                
        parameters['sampled_spectrum'] = sample_spectrum(parameters)
        
        displacement, integration_result_with_units = integrate(parameters)
        plot(displacement, integration_result_with_units)
        
        pyplot.show()

ipywidgets.interact_manual(parcel, kappa=1.28, updraft_m_s=2, n_sd=32)
IPython.display.display(output)

interactive(children=(FloatSlider(value=1.28, description='kappa', max=3.84, min=-1.28), IntSlider(value=2, de…

Output()